In [ ]:
## AWS S3 configure

In [ ]:
import os
import re


import ntpath
import boto3

from datetime import datetime, timedelta

from pathlib import Path

import pygrib
import configparser



config = configparser.ConfigParser()
config.read('/home/.aws/credentials')


s3_client = boto3.client('s3', aws_access_key_id=config['default']['aws_access_key_id'],aws_secret_access_key=config['default']['aws_secret_access_key'])


## GRIB binary selection and download single variable functions 

In [ ]:
def path_leaf(path):
    """
    Get the name of a file without any extension from given path

    Parameters
    ----------
    path : file full path with extension
    
    Returns
    -------
    str
       filename in the path without extension

    """
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)


def get_byte_range(lines,searchstring_list):
    byte_range_list=[]
    for searchstring in searchstring_list:
        expr = re.compile(searchstring)
        byte_ranges = {}
        for n, line in enumerate(lines, start=1):
            # n is the line number (starting from 1) so that when we call for 
            # `lines[n]` it will give us the next line. (Clear as mud??)
    
            # Use the compiled regular expression to search the line
            if expr.search(line):   
                # aka, if the line contains the string we are looking for...
    
                # Get the beginning byte in the line we found
                parts = line.split(':')
                rangestart = int(parts[1])
    
                # Get the beginning byte in the next line...
                if n+1 < len(lines):
                    # ...if there is a next line
                    parts = lines[n].split(':')
                    rangeend = int(parts[1])
                else:
                    # ...if there isn't a next line, then go to the end of the file.
                    rangeend = ''
    
                # Store the byte-range string in our dictionary, 
                # and keep the line information too so we can refer back to it.
                byte_ranges['start_byte']=rangestart
                byte_ranges['end_byte']=rangeend
                byte_ranges['parse_line'] = line
                byte_range_list.append(byte_ranges)
    if len(byte_range_list) ==2:
        pbyte_range={}
        pbyte_range['start_byte']=byte_range_list[0]['start_byte']
        pbyte_range['end_byte']=byte_range_list[1]['end_byte']
        pbyte_range['parse_line'] = byte_range_list[1]['parse_line']
    else:
        pbyte_range=byte_range_list[0]
    return pbyte_range
            

def s3botodownloadfile(s3keylocation, bucket,localfolder):
    """
    Download file in S3

    Parameters
    ----------
    s3keylocation : key location in s3 bucket
    bucket : s3 bucket with target contents
    localfolder : folder path in local to download the file
    
    Returns
    -------
    None

    """
    filename=path_leaf(s3keylocation)
    #filename=configz.date+'_pfz_gen_logs.txt' 
    s3_client.download_file(bucket,s3keylocation, '%s/%s' % (localfolder,filename))
    download_filepath=localfolder+filename
    return download_filepath, filename    
    



def s3_gfs_download(params):
    s3keylocation=f'gefs.{params.startdate}/{params.run}/atmos/pgrb2sp25/gep{params.ensmem}.t{params.run}z.pgrb2s.0p25.f{params.timestep}'
    s3keylocation_idx=f'{s3keylocation}.idx'
    print(s3keylocation)
    download_filepath, filename=s3botodownloadfile(s3keylocation_idx, params.awsbucketname,params.localfolder)
    with open(download_filepath) as f:
        lines = [line.rstrip() for line in f]
        
    # Search 
    if isinstance(params.searchString, list):
        searchstring_list=params.searchString
        try:
            byte_ranges=get_byte_range(lines,searchstring_list)
        except Exception as e:
            print(e)
            pass
    else:
        searchstring_list=[params.searchString]
        try:
            byte_ranges=get_byte_range(lines,searchstring_list)
        except Exception as e:
            print(e)
            pass

    gribfilename=f'gep{params.ensmem}.t{params.run}z.pgrb2s.0p25.f{params.timestep}'
    print(gribfilename)
    try:
        obj = s3_client.get_object(
            Bucket=params.awsbucketname,
            Key=s3keylocation,
            Range='bytes={}-{}'.format(byte_ranges['start_byte'], byte_ranges['end_byte']))
        with open(params.localfolder+gribfilename, 'wb') as f:
            for chunk in obj['Body'].iter_chunks(chunk_size=4096):
                f.write(chunk)
        return gribfilename
    except Exception as e:
        print(e)
        pass


## applying the functions

In [ ]:
import numpy as np
import os

hourlist=np.arange(3,243,3)
member_list=np.arange(1,22,1)

gfs_timestamp_list=[str(i).zfill(3) for i in hourlist]
gefs_member_list=[str(i).zfill(2) for i in member_list]


def foldercreator(path):
   """
    creates a folder

    Parameters
    ----------
    path : folder path
            
    Returns
    -------
    creates a folder
    """
   if not os.path.exists(path):
        os.makedirs(path)

        
class bin_create_params:
    startdate='20220915'   
    run='18'
    time='2016-04-30T06:00:00.000Z'
    folderpath='/home/hazard_data_gefs/2022082906_apcp/'
    localfolder=folderpath
    #timestep='006'
    short_name='tmp'
    awsbucketname='noaa-gefs-pds'
    test_implmentation=0
    searchString='APCP:surface:'
    #searchString='TMP:2 m above ground'
    ensmem='01'
    
params=bin_create_params()

        

for gefs_member  in gefs_member_list:
    gefs_path=f'/home/hazard_data_gefs/{params.startdate}{params.run}/'
    foldercreator(gefs_path)
    gefs_member_path=f'{gefs_path}{gefs_member}/'
    foldercreator(gefs_member_path)
    for gfs_time in gfs_timestamp_list:
        params.timestep=gfs_time
        params.ensmem=gefs_member
        params.folderpath=gefs_member_path
        params.localfolder=gefs_member_path
        s3_gfs_download(params)